In [ ]:
from konlpy.tag import Komoran
from soyclustering import SphericalKMeans
from wordcloud import WordCloud

from collections import Counter
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer
  
import matplotlib.pyplot as plt 
import nltk

## 트위터 데이터 읽기

In [ ]:
#datafilename = 'tweet/shinhan_2020-01-01_to_2020-06-13.csv' # 데이터 파일명
#datafilename = 'tweet/samsung_2020-01-01_to_2020-06-13.csv' # 데이터 파일명

#df_csv = pd.read_csv(datafilename)    # 파일 읽기

#texts = df_csv['text'].to_list() 
#df_texts = pd.DataFrame(df_csv['text'])
#df_texts.head()

# [Lab] 네이버 신한생명과 삼성생명의 블로그 글 비교

## 네이버 데이터 읽기

In [ ]:
datafilename = 'naver_api/shinhan_naver_blog.json' # 데이터 파일명
#datafilename = 'naver_api/shinhan_naver_cafe.json' # 데이터 파일명
#datafilename = 'naver_api/shinhan_naver_news.json' # 데이터 파일명

#datafilename = 'naver_api/samsung_naver_blog.json' # 데이터 파일명
#datafilename = 'naver_api/samsung_naver_cafe.json' # 데이터 파일명
#datafilename = 'naver_api/samsung_naver_news.json' # 데이터 파일명

                                               # 파일 읽기
                                               # DataFrame에서 분석 부분 추출
 


## 형태소 분석 및 저빈도 단어 제거

In [ ]:
komoran = Komoran()     # Komoran()를 활용한 형태소 분석

# 명사 추출 및 카운터 생성
noun_counter = Counter(  
    [noun for text in texts for noun in komoran.nouns(text)]
)

print('num of nouns: %d' % len(noun_counter))

In [ ]:
# 저빈도 단어 제거
noun_dict = {
    word for word, freq in noun_counter.items()
    if freq >= 2
}
len(noun_dict)

## 커스텀 토크나이저 정의

In [ ]:
# 커스텀 토크나이저 정의
def custom_tokenizer(doc):
    return [word for word in komoran.nouns(doc) if word in noun_dict]

print(custom_tokenizer(texts[0]))
print(custom_tokenizer(texts[1]))
print(custom_tokenizer(texts[2]))

## 문서 벡터 생성

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 벡터 생성을 위한 Counter 토크나이저 생성
vectorizer = CountVectorizer(
    tokenizer=custom_tokenizer,
    min_df=0.005,
    max_df=0.98
)
# 텍스트 토크나이저 적용을 통한 벡터 생성
x_sparse = vectorizer.fit_transform(texts)
print(vectorizer.vocabulary_)

In [ ]:
x_sparse.shape # 벡터 생성 결과 (벡터 크기)

In [ ]:
x_sparse.toarray() # 벡터 생성 결과

## TF-IDF 적용

In [ ]:
# TF-IDF 적용
from sklearn.feature_extraction.text import TfidfTransformer
x_tfidf = TfidfTransformer().fit_transform(x_sparse)

In [ ]:
x_tfidf.toarray() # TF-IDF 적용 결과

## 군집화 실행

In [ ]:
# K-mean 클러스터링 적용


clustersize=4   # 클러스터의 수 설정

In [ ]:
# SphericalKMeans 클러스터링 적용
#from soyclustering import SphericalKMeans

In [ ]:
                                           # K-mean 생성


                                            # K-mean 적용

# 클러스터링 결과 df_texts에 추가
df_texts['label']=labels
df_texts.info()

In [ ]:
# kmeans = SphericalKMeans(  # K-mean 생성
#    n_clusters = clustersize,
#    init = 'similar_cut',
#    verbose = True,
#    random_state = 0,
#)

## 클러스터별 단어 빈도수 확인

In [ ]:
# 군집 마다 가장 빈도수가 높은 단어 출력
for i in range(clustersize):
    lists = df_texts[df_texts['label']==i].iloc[:,0].to_list()

    textlist = []
    for text in lists:
        textlist.extend(custom_tokenizer(text))
        
    print ('\ncluster'+str(i)+': '+ str(Counter(textlist).most_common(20)))
    print(df_texts[df_texts['label']==i].iloc[0:20,0])

## 워드 클라우드로 결과 확인

In [ ]:
# 워드 클라우드 그리는 함수 정의 (빈도 고려 하지 않음)
def drawWordcloud(texts, name='wordcloud'):
    wordcloud = WordCloud(
        font_path = 'c:/windows/fonts/gulim.ttc', # 윈도우 폰트 위치
#       font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf', 
        background_color='white', colormap = 'Accent_r', width = 500, height = 500)

    noun_counter = Counter(  
        [noun for text in texts for noun in custom_tokenizer(text)]
    )
    
    wordcloud_words = wordcloud.generate_from_frequencies(noun_counter)

    # wordcloud 출력
      
        
        

    # wordcloud 저장
    fig.savefig(name+'.png')
    
    return

In [ ]:
drawWordcloud(texts) # 전체 text로 결과 출력 (빈도 고려 하지 않음)

In [ ]:
# 군집 마다 가장 빈도수가 높은 단어 출력 (빈도 고려 하지 않음)
for i in range(clustersize):
    drawWordcloud(df_texts[df_texts['label']==i].iloc[:,0].to_list(), 'cluster:'+str(i))

In [ ]:
# 워드 클라우드 그리는 함수 정의 (빈도가 높은 단어 제거)
def drawWordcloud_filter(texts, name='wordcloud'):

    wordcloud = WordCloud(
        font_path = 'c:/windows/fonts/gulim.ttc', # 윈도우 폰트 위치
#        font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf', 
        background_color='white', colormap = 'Accent_r', width = 500, height = 500)

    # wordcloud 생성
    vectorizer = CountVectorizer(
        tokenizer=custom_tokenizer,
        min_df=0.005,
        max_df=0.95
    )
    vectorizer.fit_transform(texts)
    wordcloud_words = wordcloud.generate_from_frequencies(vectorizer.vocabulary_)

    # wordcloud 출력
    fig = plt.figure(figsize=(10, 10))  # 그림 크기 설정
    plt.title(name)
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis('off')
    plt.show()

    # wordcloud 저장
    fig.savefig(name+'.png')
    
    return

In [ ]:
# 전체 text로 결과 출력 (빈도가 높은 단어 제거)
drawWordcloud_filter(texts) 

In [ ]:
# 군집 마다 가장 빈도수가 높은 단어 출력 (빈도가 높은 단어 제거)
for i in range(clustersize):
    drawWordcloud_filter(df_texts[df_texts['label']==i].iloc[:,0].to_list(), 'cluster:'+str(i))